<a href="https://colab.research.google.com/github/vikasdo/BikeHub/blob/main/LinearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from wordcloud import WordCloud, STOPWORDS
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, accuracy_score
from math import sqrt

In [2]:
!wget https://github.com/vikasdo/BikeHub/blob/main/day.csv

--2024-11-19 17:01:40--  https://github.com/vikasdo/BikeHub/blob/main/day.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘day.csv’

day.csv                 [ <=>                ] 374.71K  --.-KB/s    in 0.02s   

2024-11-19 17:01:40 (20.5 MB/s) - ‘day.csv’ saved [383707]

